In [142]:
X = torch.Tensor{{1,0}, {1,1}, {0,1}}
Y = torch.Tensor{{1,0,0}, {0,1,0}, {0,0,1}}
print(X)
print(Y)

cc = torch.zeros(1, Y:size(1))
fc = torch.zeros(Y:size(1), X:size(2))
lp = torch.zeros(Y:size(1), X:size(2))
print(cc)
print(fc)

 1  0
 1  1
 0  1
[torch.DoubleTensor of size 3x2]

 1  0  0
 0  1  0
 0  0  1
[torch.DoubleTensor of size 3x3]

 0  0  0
[torch.DoubleTensor of size 1x3]

 0  0
 0  0
 0  0
[torch.DoubleTensor of size 3x2]



In [143]:
function counts(X, Y)
    fc:add(Y:t()*(X))
    cc:add(Y:sum(1))
end

In [144]:
function update_feature_log_prob(fc)
    smoothed_fc = torch.add(fc, 1)
    smoothed_cc = smoothed_fc:sum(2)
    --ones = torch.ones(smoothed_cc:size(1), smoothed_fc:size(2))
    print(smoothed_fc)
    print(smoothed_cc)
    for i=1,smoothed_cc:size(1) do
        smoothed_fc:select(1,i):add(smoothed_fc:select(1,i))
    end
    print(smoothed_fc)
end   

In [145]:
counts(X,Y)
print(fc)
print(cc)
update_feature_log_prob(fc)

[string "function update_feature_log_prob(fc)..."]:8: internal error in __mul: no metatable
stack traceback:
	[C]: in function '__mul'
	[string "function update_feature_log_prob(fc)..."]:8: in function 'update_feature_log_prob'
	[string "counts(X,Y)..."]:4: in main chunk
	[C]: in function 'xpcall'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/Vincent/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010805cbd0: 

 1  0
 1  1
 0  1
[torch.DoubleTensor of size 3x2]

 1  1  1
[torch.DoubleTensor of size 1x3]

 2  1
 2  2
 1  2
[torch.DoubleTensor of size 3x2]

 3
 4
 3
[torch.DoubleTensor of size 3x1]



In [80]:
for i=1,2 do
    fc:select(2, i):add(cc * -1)
end

In [105]:
x = fc:sum(2)

In [114]:
x:select(1, 3)

 1
[torch.DoubleTensor of size 1]



In [133]:
fc:select(1,3):add(-1)

-1
 0
[torch.DoubleTensor of size 2]



In [131]:
fc

 0 -1
 1  1
 0  1
[torch.DoubleTensor of size 3x2]

